## Регрессия поломок

Устанавливаем три библиотеки Python:

catboost: Для построения модели машинного обучения.

featuretools: Для автоматического создания новых признаков из данных.

optuna: Для поиска наилучших настроек для модели.

Эти библиотеки используются вместе для создания модели для предсказания поломок автомобилей на основе предоставленных данных.

In [1]:
!pip install catboost -q
!pip install featuretools -q
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.9/587.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.2/215.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00


Отключаем вывод предупреждений о потенциальных проблемах, чтобы не загромождать вывод программы.

In [2]:
import warnings

warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)

In [3]:
import featuretools as ft
import numpy as np
import optuna
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from woodwork.logical_types import Age, Categorical, Datetime

In [4]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

Этот код подготавливает данные и параметры для создания модели, которая будет предсказывать "target_class" автомобиля на основе других его характеристик.

In [5]:
N_SPLITS = 3
N_TRIALS = 50
RANDOM_SEED = 42
SEARCH_BEST_PARAMS = False

CAT_FEATURES = ["model", "car_type", "fuel_type"]
TARGET_COL = "target_reg"

FEATURES_TO_DROP = ["car_id", "target_reg", "deviation_normal_count"]

## Загрузка данных

Загружаем данные о машинах из 5 файлов. Затем разделяет данные на обучающую и тестовую выборки. Удаляет столбец с ответом (target_class) из обучающей выборки и объединяет обучающую и тестовую выборки для дальнейшей обработки.

В итоге, код подготавливает данные для создания модели, которая будет предсказывать поломки автомобилей.

In [6]:
car_info_train = pd.read_csv("car_train.csv")
car_info_test = pd.read_csv("car_test.csv")
rides_info = pd.read_csv("rides_info.csv")
driver_info = pd.read_csv("driver_info.csv")
fix_info = pd.read_csv("fix_info.csv")

train_cars = car_info_train["car_id"]
test_cars = car_info_test["car_id"]
y_train = car_info_train[TARGET_COL]

car_info_train = car_info_train.drop(columns=[TARGET_COL, "target_class"])
all_data = pd.concat([car_info_train, car_info_test], ignore_index=True)

## Feature Engineering

Создаем новые признаки (фич) из данных, чтобы улучшить работу модели машинного обучения, которая будет предсказывать поломки автомобилей.

Что делает код:

1. Объединяет данные: Создает связи между разными таблицами (информация о машинах, поездках, водителях, ремонтах) на основе общих столбцов (например, car_id).
2. Создает новые признаки: Автоматически генерирует новые признаки, комбинируя информацию из связанных таблиц. Например, может создать признак "среднее количество поездок в месяц для данной модели машины".
3. Очищает признаки: Удаляет ненужные признаки, которые не несут полезной информации (например, признаки, имеющие только одно значение).
4. Готовит данные для модели: Разделяет данные на обучающую и тестовую выборки, удаляет ненужные столбцы и обновляет список категориальных признаков.

В результате: код создает новые, потенциально полезные признаки, которые помогут модели машинного обучения лучше предсказывать поломки автомобилей.

Ключевые моменты:

EntitySet: Контейнер, который хранит связанные таблицы.

ft.dfs: Функция, которая автоматически генерирует новые признаки.

CAT_FEATURES: Список категориальных признаков.

X_train, X_test: Матрицы признаков для обучения и тестирования модели.

In [7]:
# Создаём отношения между источниками данных
es = ft.EntitySet(id="car_data")

es = es.add_dataframe(
    dataframe_name="cars",
    dataframe=all_data,
    index="car_id",
    logical_types={
        "car_type": Categorical,
        "fuel_type": Categorical,
        "model": Categorical,
    },
)

es = es.add_dataframe(
    dataframe_name="rides",
    dataframe=rides_info.drop(["ride_id"], axis=1),
    index="index",
    time_index="ride_date",
)

es = es.add_dataframe(
    dataframe_name="drivers",
    dataframe=driver_info,
    index="user_id",
    logical_types={
        "sex": Categorical,
        "first_ride_date": Datetime,
        "age": Age,
    },
)

es = es.add_dataframe(
    dataframe_name="fixes",
    dataframe=fix_info,
    index="index",
    logical_types={
        "work_type": Categorical,
        "worker_id": Categorical,
    },
)

es = es.add_relationship("cars", "car_id", "rides", "car_id")
es = es.add_relationship("drivers", "user_id", "rides", "user_id")
es = es.add_relationship("cars", "car_id", "fixes", "car_id")

# Генерируем фичи
all_data, _ = ft.dfs(
    entityset=es,
    target_dataframe_name="cars",
    max_depth=2,
)

# Удаляем константные фичи
all_data = ft.selection.remove_single_value_features(all_data)

train_data = all_data.loc[train_cars].reset_index()
test_data = all_data.loc[test_cars].reset_index()

X_train = train_data.drop(columns=FEATURES_TO_DROP, errors="ignore")
X_test = test_data.drop(columns=FEATURES_TO_DROP, errors="ignore")

# Обновляем список категориальных фичей
CAT_FEATURES += [col for col in X_train if col.startswith("MODE")]

In [8]:
X_train.head()

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,MAX(rides.deviation_normal),MAX(rides.distance),MAX(rides.rating),MAX(rides.refueling),MAX(rides.ride_cost),MAX(rides.ride_duration),MAX(rides.speed_avg),MAX(rides.speed_max),MAX(rides.stop_times),MAX(rides.user_ride_quality),MEAN(rides.deviation_normal),MEAN(rides.distance),MEAN(rides.rating),MEAN(rides.refueling),MEAN(rides.ride_cost),MEAN(rides.ride_duration),MEAN(rides.speed_avg),MEAN(rides.speed_max),MEAN(rides.stop_times),MEAN(rides.user_ride_quality),MIN(rides.deviation_normal),MIN(rides.distance),MIN(rides.rating),MIN(rides.ride_cost),MIN(rides.ride_duration),MIN(rides.speed_avg),MIN(rides.speed_max),MIN(rides.user_ride_quality),SKEW(rides.deviation_normal),SKEW(rides.distance),SKEW(rides.rating),SKEW(rides.refueling),SKEW(rides.ride_cost),SKEW(rides.ride_duration),SKEW(rides.speed_avg),SKEW(rides.speed_max),SKEW(rides.stop_times),SKEW(rides.user_ride_quality),STD(rides.deviation_normal),STD(rides.distance),STD(rides.rating),STD(rides.refueling),STD(rides.ride_cost),...,SUM(rides.ride_cost),SUM(rides.ride_duration),SUM(rides.speed_avg),SUM(rides.speed_max),SUM(rides.stop_times),SUM(rides.user_ride_quality),COUNT(fixes),MAX(fixes.destroy_degree),MAX(fixes.work_duration),MEAN(fixes.destroy_degree),MEAN(fixes.work_duration),MIN(fixes.work_duration),MODE(fixes.work_type),MODE(fixes.worker_id),NUM_UNIQUE(fixes.work_type),NUM_UNIQUE(fixes.worker_id),SKEW(fixes.destroy_degree),SKEW(fixes.work_duration),STD(fixes.destroy_degree),STD(fixes.work_duration),SUM(fixes.destroy_degree),SUM(fixes.work_duration),MAX(rides.drivers.age),MAX(rides.drivers.user_rating),MAX(rides.drivers.user_rides),MAX(rides.drivers.user_time_accident),MEAN(rides.drivers.age),MEAN(rides.drivers.user_rating),MEAN(rides.drivers.user_rides),MEAN(rides.drivers.user_time_accident),MIN(rides.drivers.age),MIN(rides.drivers.user_rating),MIN(rides.drivers.user_rides),MIN(rides.drivers.user_time_accident),MODE(rides.DAY(ride_date)),MODE(rides.MONTH(ride_date)),MODE(rides.WEEKDAY(ride_date)),MODE(rides.drivers.sex),SKEW(rides.drivers.age),SKEW(rides.drivers.user_rating),SKEW(rides.drivers.user_rides),SKEW(rides.drivers.user_time_accident),STD(rides.drivers.age),STD(rides.drivers.user_rating),STD(rides.drivers.user_rides),STD(rides.drivers.user_time_accident),SUM(rides.drivers.age),SUM(rides.drivers.user_rating),SUM(rides.drivers.user_rides),SUM(rides.drivers.user_time_accident)
0,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,0.001,1.849349e+06,9.44,0.0,523483.0,37392.0,77.0,180.855726,20.0,11.035871,-0.120391,69777.646008,4.737759,0.0,20106.873563,1635.770115,44.660920,87.183965,3.620690,-0.901190,-9.000,71.606506,0.1,44.0,4.0,25.0,38.0,-10.501738,-8.207991,4.697610,0.129744,0.0,4.733686,4.508536,0.400501,0.845063,1.566092,-0.080287,0.861750,276803.751158,2.001670,0.0,77483.240255,...,3498596.0,284624.0,7771.0,14734.090155,630.0,-156.807066,35,9.0,56.0,3.048571,26.657143,7.0,reparking,LR,4,33,0.835907,0.826462,2.732847,10.171884,106.7,933.0,62.0,9.8,2626.0,77.0,33.511494,8.229885,828.034483,17.724138,18.0,6.2,5.0,0.0,1,1,2,1,0.268286,-0.170288,0.751130,1.813293,10.109652,0.611473,546.505545,17.294174,5831.0,1432.0,144078.0,2056.0
1,VW Polo VI,economy,petrol,3.90,2015,78218,2021,47.673,2.762119e+06,10.00,0.0,609068.0,38450.0,88.0,187.862734,3.0,32.610351,6.050011,103672.947347,4.480517,0.0,26813.614943,2223.178161,49.862069,89.474427,0.833333,14.018105,-25.088,152.315513,0.0,74.0,7.0,25.0,39.0,0.437053,0.415372,4.351280,0.337138,0.0,4.166274,3.739653,0.838893,0.801942,0.768856,0.387577,16.102649,379729.690043,2.829132,0.0,98135.975960,...,4665569.0,386833.0,8676.0,15568.550271,145.0,2439.150253,35,10.0,48.0,2.917143,24.942857,4.0,reparking,YH,5,34,0.997276,-0.296841,2.707233,8.574733,102.1,873.0,57.0,9.8,2821.0,23.0,34.988506,7.988506,924.804598,6.965517,18.0,6.7,7.0,0.0,1,1,2,0,-0.023846,0.236130,0.555751,0.749350,9.592259,0.574089,585.134416,4.911820,6088.0,1390.0,160916.0,1212.0
2,Renault S

In [9]:
X_test.head()

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,MAX(rides.deviation_normal),MAX(rides.distance),MAX(rides.rating),MAX(rides.refueling),MAX(rides.ride_cost),MAX(rides.ride_duration),MAX(rides.speed_avg),MAX(rides.speed_max),MAX(rides.stop_times),MAX(rides.user_ride_quality),MEAN(rides.deviation_normal),MEAN(rides.distance),MEAN(rides.rating),MEAN(rides.refueling),MEAN(rides.ride_cost),MEAN(rides.ride_duration),MEAN(rides.speed_avg),MEAN(rides.speed_max),MEAN(rides.stop_times),MEAN(rides.user_ride_quality),MIN(rides.deviation_normal),MIN(rides.distance),MIN(rides.rating),MIN(rides.ride_cost),MIN(rides.ride_duration),MIN(rides.speed_avg),MIN(rides.speed_max),MIN(rides.user_ride_quality),SKEW(rides.deviation_normal),SKEW(rides.distance),SKEW(rides.rating),SKEW(rides.refueling),SKEW(rides.ride_cost),SKEW(rides.ride_duration),SKEW(rides.speed_avg),SKEW(rides.speed_max),SKEW(rides.stop_times),SKEW(rides.user_ride_quality),STD(rides.deviation_normal),STD(rides.distance),STD(rides.rating),STD(rides.refueling),STD(rides.ride_cost),...,SUM(rides.ride_cost),SUM(rides.ride_duration),SUM(rides.speed_avg),SUM(rides.speed_max),SUM(rides.stop_times),SUM(rides.user_ride_quality),COUNT(fixes),MAX(fixes.destroy_degree),MAX(fixes.work_duration),MEAN(fixes.destroy_degree),MEAN(fixes.work_duration),MIN(fixes.work_duration),MODE(fixes.work_type),MODE(fixes.worker_id),NUM_UNIQUE(fixes.work_type),NUM_UNIQUE(fixes.worker_id),SKEW(fixes.destroy_degree),SKEW(fixes.work_duration),STD(fixes.destroy_degree),STD(fixes.work_duration),SUM(fixes.destroy_degree),SUM(fixes.work_duration),MAX(rides.drivers.age),MAX(rides.drivers.user_rating),MAX(rides.drivers.user_rides),MAX(rides.drivers.user_time_accident),MEAN(rides.drivers.age),MEAN(rides.drivers.user_rating),MEAN(rides.drivers.user_rides),MEAN(rides.drivers.user_time_accident),MIN(rides.drivers.age),MIN(rides.drivers.user_rating),MIN(rides.drivers.user_rides),MIN(rides.drivers.user_time_accident),MODE(rides.DAY(ride_date)),MODE(rides.MONTH(ride_date)),MODE(rides.WEEKDAY(ride_date)),MODE(rides.drivers.sex),SKEW(rides.drivers.age),SKEW(rides.drivers.user_rating),SKEW(rides.drivers.user_rides),SKEW(rides.drivers.user_time_accident),STD(rides.drivers.age),STD(rides.drivers.user_rating),STD(rides.drivers.user_rides),STD(rides.drivers.user_time_accident),SUM(rides.drivers.age),SUM(rides.drivers.user_rating),SUM(rides.drivers.user_rides),SUM(rides.drivers.user_time_accident)
0,Skoda Rapid,economy,petrol,4.80,2013,42269,2019,36.661,3.021921e+06,8.91,0.0,336198.0,37356.0,100.0,195.454152,3.0,23.362400,16.664374,80893.068947,3.746207,0.0,14495.442529,1316.275862,51.482759,100.987907,0.827586,9.229144,-11.495,19.590090,0.10,19.0,3.0,25.0,38.446584,-10.297021,-0.593285,5.930494,0.184308,0.0,4.229911,4.641263,0.667228,0.453493,0.624510,-0.428845,12.162105,336344.827625,2.326693,0.0,51964.626097,...,2522207.0,229032.0,8958.0,17571.895793,144.0,1605.871135,35,8.0,49.0,2.765714,23.485714,4.0,reparking,BB,4,35,0.750094,0.312615,2.396682,8.125383,96.8,822.0,57.0,9.8,3287.0,25.0,33.706897,7.894253,849.212644,9.568966,18.0,5.8,10.0,0.0,1,1,2,1,0.166132,0.124766,0.735757,0.711919,10.280475,0.632886,580.542915,4.722760,5865.0,1373.6,147763.0,1665.0
1,Renault Sandero,standart,petrol,4.32,2015,90014,2016,0.085,2.279221e+06,10.00,0.0,629499.0,39644.0,87.0,181.538685,4.0,30.378596,-0.082115,113240.814175,4.318966,0.0,27256.063218,2215.459770,50.356322,92.576238,0.752874,12.871315,-7.925,38.730162,0.00,9.0,2.0,25.0,38.000000,-1.288132,-8.796293,3.790663,0.398829,0.0,4.062327,3.697281,0.728297,0.453788,0.948945,0.579107,0.740807,400328.058724,2.512479,0.0,98130.403053,...,4742555.0,385490.0,8762.0,15737.960397,131.0,2239.608770,35,10.0,48.0,3.657143,25.428571,4.0,reparking,XX,3,34,0.640624,0.171686,2.939616,10.757448,128.0,890.0,62.0,9.8,3207.0,24.0,34.850575,7.998276,857.528736,6.896552,18.0,6.3,1.0,0.0,1,1,2,0,-0.002022,-0.148100,1.238131,0.932452,10.115551,0.617048,600.837062,4.874220,6064.0,1391.7,149210.0,1200

In [16]:
# Подготовка данных для CatBoost
X_train_catboost = X_train.copy()
X_test_catboost = X_test.copy()
for cat_feature in CAT_FEATURES:
    X_train_catboost[cat_feature] = X_train_catboost[cat_feature].astype(str)
    X_test_catboost[cat_feature] = X_test_catboost[cat_feature].astype(str)


In [22]:
from sklearn.model_selection import train_test_split

# Разделение данных
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_catboost,  # Входные данные
    y_train,           # Целевой признак
    test_size=0.2,     # Размер тестовой выборки (например, 20%)
    random_state=42    # Для воспроизводимости
)

print(f"Размер тренировочных данных: {X_train.shape}")
print(f"Размер валидационных данных: {X_valid.shape}")


Размер тренировочных данных: (1869, 109)
Размер валидационных данных: (468, 109)


In [23]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

# Задаем параметры модели
params = {
    "learning_rate": 0.1,       # Шаг обучения
    "depth": 6,                 # Глубина дерева
    "l2_leaf_reg": 3.0,         # Регуляризация
    "random_seed": 42,          # Фиксация случайности
    "eval_metric": "MAE",       # Метрика качества
    "verbose": 100,             # Логи каждые 100 итераций
    "cat_features": CAT_FEATURES,         # Категориальные признаки, если есть
    "od_type": "Iter",          # Тип остановки по раннему прекращению
    "od_wait": 50,              # Количество итераций без улучшения для остановки
}

# Инициализация модели
model = CatBoostRegressor(**params)

model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    use_best_model=True
)

# Предсказания на тестовой выборке
y_pred = model.predict(X_valid)

# Оценка качества модели
mae = mean_absolute_error(y_valid, y_pred)
print(f"Mean Absolute Error (MAE) на тестовой выборке: {mae}")

# Сохранение модели (опционально)
model.save_model("catboost_regressor_model.cbm", format="cbm")


0:	learn: 12.5768374	test: 13.2367034	best: 13.2367034 (0)	total: 33.4ms	remaining: 33.4s
100:	learn: 4.1120709	test: 5.2284187	best: 5.2284187 (100)	total: 2.66s	remaining: 23.7s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 5.205052981
bestIteration = 109

Shrink model to first 110 iterations.
Mean Absolute Error (MAE) на тестовой выборке: 5.205053961135221


In [25]:
from catboost import CatBoostRegressor

# Загружаем модель из файла
loaded_model = CatBoostRegressor()
loaded_model.load_model("catboost_regressor_model.cbm")

# Предсказания на тестовых данных
y_pred_test = loaded_model.predict(X_test_catboost)

# Добавление предсказаний к тестовым данным
test_data["target_reg"] = y_pred_test

# Сохранение результата в файл
test_data[["car_id", "target_reg"]].to_csv("submission_r.csv", index=False)

print("Предсказания успешно выполнены и сохранены в 'submission_r.csv'.")


Предсказания успешно выполнены и сохранены в 'submission_r.csv'.
